In [1]:
import geopandas as gp
import pandas as pd
import xarray as xr
import rioxarray
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import seaborn as sb
import sys
from importlib import reload
sys.path.append('..')


In [2]:
HySpexPth = Path("/Volumes/FIREICE/fihyper/cwaigl/")
subpath = "03_products/cropped_masked_final/"
filepatt = "VNIR_SWIR_rad_geo_atm_bcor_crop.bsq"

In [3]:
selected_band_idx = np.array([284,
 1,
 235,
 346,
 152,
 311,
 49,
 238,
 364,
 119,
 87,
 250,
 409,
 117,
 361,
 189,
 54,
 303,
 218,
 85,
 252,
 149,
 362,
 173,
 428,
 14,
 217,
 260,
 104,
 368,
 96,
 421,
 9,
 218,
 330,
 107,
 431,
 41,
 253,
 187,
 349,
 92,
 312,
 33,
 218,
 407,
 157,
 370,
 27,
 217])

In [4]:
def get_datafile(row, smooth=True):
    fpth = HySpexPth / f"{row.fileprefix.replace('-', '_')}" / subpath / f"{row.fileprefix}_{row.flightline}_{filepatt}"
    return fpth

def get_singlefilesampledf(labelrow, sampledf=None):
    return sampledf[(sampledf.fileprefix==labelrow.fileprefix) & (sampledf.flightline==labelrow.flightline)]

def get_dataset(fpth):
    testdata = xr.open_dataset(fpth, engine="rasterio")
    testdata.attrs['long_name'] = "spectral reflectance, %*100"
    testdata.attrs['name'] = "spectral reflectance, %*100"
    testdata['band_data'].attrs['long_name']  =  "spectral reflectance, %*100"
    return testdata

def get_spectra_fromsingle(samplerow, dataset='dummy'):
    pt = samplerow.geometry
    spectrum = dataset.band_data.sel(x=pt.x, y=pt.y, method='nearest').values[:-10]
    return spectrum

def get_spectra(labelrow, sampledf=None):
    results = []
    subsubsample = get_singlefilesampledf(labelrow, sampledf)
    fn = get_datafile(labelrow)
    print(f"working on {fn}")
    testdata = get_dataset(fn)
    print("Got dataset opened") 
    for idx, row in subsubsample.iterrows():
        print(f"   working on {idx}, {row.species} at {row.geometry}") 
        spectrum = get_spectra_fromsingle(row, dataset=testdata)
        dictitem = {
            'fid': idx,
            'species': row.species,
            'event': row.fileprefix,
            'flightline': row.flightline,
            'vegclass': row.vegclass,
            'context': row.context,
            'spectrum': spectrum,
            }
        results.append(dictitem)
    return results
    testdata.close()

In [5]:
vegpath = Path("/Volumes/FIREICE/fiboreal/cwaigl/2021-HySpex_fuels/GIS/veg_locations/") 
hex20_BC_2020 = "BC_hextest01.gpkg"

In [7]:
hex20_df = gp.read_file(vegpath / hex20_BC_2020)
hex20_df.rename(columns={'flighline': 'flightline'}, inplace=True)
hex20_df['fileprefix'] = '20200830-BC'
hex20_df['flightline'] = '05'
hex20_df

,id,left,top,right,bottom,geometry,fileprefix,flightline
0,122073,438134.325310,7.178334e+06,438157.41932,7.178314e+06,"POLYGON ((438134.325 7178323.741, 438140.099 7...",20200830-BC,05
1,122074,438134.325310,7.178314e+06,438157.41932,7.178294e+06,"POLYGON ((438134.325 7178303.741, 438140.099 7...",20200830-BC,05
2,122075,438134.325310,7.178294e+06,438157.41932,7.178274e+06,"POLYGON ((438134.325 7178283.741, 438140.099 7...",20200830-BC,05
3,122076,438134.325310,7.178274e+06,438157.41932,7.178254e+06,"POLYGON ((438134.325 7178263.741, 438140.099 7...",20200830-BC,05
4,122077,438134.325310,7.178254e+06,438157.41932,7.178234e+06,"POLYGON ((438134.325 7178243.741, 438140.099 7...",20200830-BC,05
...,...,...,...,...,...,...,...,...
402,131032,438498.055979,7.178064e+06,438521.14999,7.178044e+06,"POLYGON ((438498.056 7178053.741, 438503.829 7...",20200830-BC,05
403,131033,438498.055979,7.178044e+06,438521.14999,7.178024e+06,"POLYGON ((438498.056 7178033.741, 438503.829 7...",20200830-BC,05
404,131034,438498.055979,7.178024e+06,438521.14999,7.178004e+06,"POLYGON ((438498.056 7178013.741, 438503.829 7...",20200830-BC,05
405,131035,438498.055979,7.178004e+06,438521.14999,7.177984e+06,"POLYGON ((438498.056 7177993.741, 438503.829 7...",20200830-BC,05


In [10]:
labeldf = hex20_df[['fileprefix', 'flightline']].drop_duplicates()
samples = []
for _, labelrow in labeldf.iterrows():
    subsubsample = get_singlefilesampledf(labelrow, hex20_df)
    fn = get_datafile(labelrow)
    print(f"working on {fn}")

    for idx, row in subsubsample.iterrows():
        print(f"   working on {idx}") 
        try:
            clipped = rioxarray.open_rasterio(fn, masked=True).rio.clip([row.geometry], from_disk=True)
        except ValueError:
            continue
        stacked = clipped.sel(band=(selected_band_idx[:-10] + 1)).stack(desired=['x', 'y'])
        dictelem = {
            'data_means': stacked.mean(axis=1).data,
            'data_stds': stacked.std(axis=1).data
        }
        samples.append(dictelem)

samples
                                     


working on /Volumes/FIREICE/fihyper/cwaigl/20200830_BC/03_products/cropped_masked_final/20200830-BC_05_VNIR_SWIR_rad_geo_atm_bcor_crop.bsq
   working on 0
   working on 1
   working on 2
   working on 3
   working on 4
   working on 5
   working on 6
   working on 7
   working on 8
   working on 9
   working on 10
   working on 11
   working on 12
   working on 13
   working on 14
   working on 15
   working on 16
   working on 17
   working on 18
   working on 19
   working on 20
   working on 21
   working on 22
   working on 23
   working on 24
   working on 25
   working on 26
   working on 27
   working on 28
   working on 29
   working on 30
   working on 31
   working on 32
   working on 33
   working on 34
   working on 35
   working on 36
   working on 37
   working on 38
   working on 39
   working on 40
   working on 41
   working on 42
   working on 43
   working on 44
   working on 45
   working on 46
   working on 47
   working on 48
   working on 49
   working on 50
   w

[{'data_means': array([1275.85549133,   43.42196532, 3493.36416185,  185.07803468,
         3028.4132948 , 1573.23988439,  460.40751445, 3335.66763006,
          484.39884393, 2872.51734104,  274.22543353, 2062.07803468,
          722.89884393, 2861.74277457,  152.5867052 , 3702.66184971,
          409.03179191, 1708.30924855, 3132.7716763 ,  231.98843931,
         1721.80635838, 3140.96820809,  223.57514451, 3234.47398844,
          543.41907514,   95.65317919, 3132.97398844,  891.25433526,
         2537.79479769,  520.57514451, 1471.00578035,  566.63294798,
           89.60982659, 3132.7716763 ,  753.20809249, 2658.12427746,
          656.44508671,  438.73410405, 1563.44797688, 3550.37861272]),
  'data_stds': array([ 341.82331057,   38.54726073,  839.10005831,   53.68590124,
         1157.68014229,  409.51815437,  234.83187653,  797.84718063,
         1573.88098514, 1137.48245289,  133.12433252,  499.796678  ,
          226.07222655, 1138.04909647,   41.55210876,  932.58200129,
     